# Lecture 9: Frequency Containment Reserve (FCR)

This notebook explores the frequency containment reserve for grid scale battery energy storage systems (BESS).

A techno-economic analysis is performed with the help of [simses](https://gitlab.lrz.de/open-ees-ses/simses) a simulation framework for stationary energy storage systems. The frequency profile is taken from the TSO 50Hertz, the data is from January 2019.

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from simses.main import SimSES
from configparser import ConfigParser

In [ ]:
pd.options.plotting.backend = "plotly"
# template = "plotly_dark"
template = "plotly_white"

## Simulation parameters

In [ ]:
power_fcr = 1.0e6 # W
power_idm = 0.25e6 # W
capacity  = 1.25e6 # Wh
sim_params = f"""
[GENERAL]
START = 2014-01-01 00:00:00
END = 2014-01-31 23:59:59
TIME_STEP = 60
LOOP = 1

[ENERGY_MANAGEMENT]
STRATEGY = FcrIdmRechargeStacked
POWER_FCR = {power_fcr}
POWER_IDM = {power_idm}
FCR_RESERVE = 0.25

[BATTERY]
START_SOC = 0.5
MIN_SOC = 0.0
MAX_SOC = 1

[STORAGE_SYSTEM]
; Configuration of the AC storage system:
; Format: AC-system name, max AC power in W, DC voltage level in V, ACDC converter name, housing name, HVAC name
STORAGE_SYSTEM_AC =
    system_1,{power_fcr + power_idm},333,notton,no_housing,no_hvac

; Configuration of the AC/DC converter:
; Format: ACDC converter name, converter type, optional: number of converters
ACDC_CONVERTER =
    notton,NottonAcDcConverter

; Configuration of the DC storage system. Every AC system must have at least 1 DC system
; Format: AC-system name, DCDC converter name, storage technology name
STORAGE_SYSTEM_DC =
   system_1,no_loss,lfp

; Configuration of the DCDC converter
; Format: DCDC converter name, converter type, [efficiency]
DCDC_CONVERTER =
    no_loss,NoLossDcDcConverter

; Configuration of the storage technology.
; Format: storage technology name, energy in Wh, technology type, [technology specific parameters]
STORAGE_TECHNOLOGY =
    lfp,{capacity},lithium_ion,SonyLFP

[PROFILE]
TECHNICAL_PROFILE_DIR = {os.path.abspath("../data")}
FREQUENCY_PROFILE = simses_frequency_profile_2014

"""
sim_config = ConfigParser()
sim_config.read_string(sim_params)

In [ ]:
path = os.path.abspath("..")
result_path = os.path.join(path, "simses_results").replace("\\", "/") + "/"

## Run simulation

In [ ]:
simses = SimSES(path=result_path, name="fcr", simulation_config=sim_config)

In [ ]:
simses.run()

## Results and analysis

In [ ]:
# results path
results = os.path.join(result_path, "fcr")
latest = os.listdir(results)[-1]
results = os.path.join(results, latest).replace("\\", "/")

In [ ]:
df_sys = pd.read_csv(results + "/SystemState.csv.gz").drop_duplicates("Time in s")
df_ems = pd.read_csv(results + "/EnergyManagementState.csv.gz")
df_lis = pd.read_csv(results + "/LithiumIonState.csv.gz")

In [ ]:
# f = pd.read_csv("../data/simses_frequency_profile_50hz201901.csv", header=None)[0]
f = pd.read_csv("../data/simses_frequency_profile_2014.csv", header=None)[0]

In [ ]:
df = pd.DataFrame(
    data = {
        "f": f.values,
        "power": -df_sys["AC_P_delivered in W"].values,
        "soc": df_sys["SOC in p.u."].values,
    },
    index = pd.date_range(start="2014-01-01", end="2014-01-31 23:59", freq="1Min")
)

In [ ]:
def plot_fcr(df, inverse_power=False, show_deadband=False, **kwargs):
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.update_layout(height=500, **kwargs)

    fig.add_trace(go.Scatter(x=df.index, y=df["f"], name="f"))
    fig.update_yaxes(title="Frequency [Hz]")
    
    fig.add_trace(go.Scatter(x=df.index, y=df["power"], name="power"), secondary_y=True)
    fig.update_yaxes(title="Power [MW]", secondary_y=True)

    if show_deadband:
        fdb = 10.0e-3
        fig.add_hline(y = 50 - fdb, opacity=0.5, line_dash="dot")
        fig.add_hline(y = 50 + fdb, opacity=0.5, line_dash="dot")
    
    f_max = 0.2 # max(abs(df["f"] - 50))
    fig.update_yaxes(range=[50-f_max, 50+f_max])

    p_max = 1.0e6 # max(abs(df["power"]))
    fig.update_yaxes(range=[-p_max, p_max], secondary_y=True)
    
    if inverse_power:
        fig.update_yaxes(range=[p_max, -p_max], secondary_y=True)

    return fig

In [ ]:
plot_fcr(df, show_deadband=True, inverse_power=False, template=template)

In [ ]:
df["soc"].plot(template=template, labels={"value": "SOC in p.u.", "index": "Time"}).update_yaxes(range=[0,1])

In [ ]:
df["soc"].plot.hist(template=template, labels={"value": "SOC in p.u.", "index": "Time"})